In [1]:
import time
import pandas as pd
import numpy as np

train_agg = pd.read_csv('./input/train_agg.csv', sep='\t')
test_agg = pd.read_csv('./input/test_agg.csv', sep='\t')
agg = pd.concat([train_agg, test_agg], copy=False)

train_log = pd.read_csv('./input/train_log.csv', sep='\t')
test_log = pd.read_csv('./input/test_log.csv', sep='\t')
log = pd.concat([train_log, test_log], copy=False)

log['EVT_LBL_1'] = log['EVT_LBL'].apply(lambda x: x.split('-')[0])
log['EVT_LBL_2'] = log['EVT_LBL'].apply(lambda x: x.split('-')[1])
log['EVT_LBL_3'] = log['EVT_LBL'].apply(lambda x: x.split('-')[2])
log['OCC_TIM_UNI'] = log['OCC_TIM'].apply(lambda x: time.mktime(time.strptime(x, "%Y-%m-%d %H:%M:%S")))
log = log.sort_values(by=['USRID', 'OCC_TIM'])

train_flg = pd.read_csv('./input/train_flg.csv', sep='\t')
test_flg = pd.read_csv('./input/submit_sample.csv', sep='\t')
test_flg['FLAG'] = -1
del test_flg['RST']

flg = pd.concat([train_flg, test_flg], copy=False)
data = pd.merge(agg, flg, on=['USRID'], how='left', copy=False)

In [2]:
def get_year(x):
    return int(x.split('-')[0])

def get_month(x):
    return int(x.split('-')[1])

def get_dayofmonth(x):
    str = (x.split('-')[2]).split(' ')[0]
    return int(str)

def get_hour(x):
    str = ((x.split('-')[2]).split(' ')[1]).split(':')[0]
    return int(str)

def get_minute(x):
    str = ((x.split('-')[2]).split(' ')[1]).split(':')[1]
    return int(str)

def get_second(x):
    str = ((x.split('-')[2]).split(' ')[1]).split(':')[2]
    return int(str)

In [3]:
log['YEAR'] = log['OCC_TIM'].apply(lambda x: get_year(x))
log['MONTH'] = log['OCC_TIM'].apply(lambda x: get_month(x))
log['DOM'] = log['OCC_TIM'].apply(lambda x: get_dayofmonth(x))
log['HOUR'] = log['OCC_TIM'].apply(lambda x: get_hour(x))
log['MINUTE'] = log['OCC_TIM'].apply(lambda x: get_minute(x))
log['SECOND'] = log['OCC_TIM'].apply(lambda x: get_second(x))
log['TIM_STP'] = log['OCC_TIM'].apply(lambda x: pd.Timestamp(get_year(x), get_month(x), get_dayofmonth(x), get_hour(x), get_minute(x), get_second(x)))
log['DOW'] = log['TIM_STP'].apply(lambda x: (x.dayofweek + 1))

In [4]:
log['next_time'] = log.groupby(by=['USRID'])['OCC_TIM_UNI'].diff(-1).apply(np.abs)

In [5]:
def encoding_bin(x):
    if x > 0:
        return 1
    else:
        return 0

data['V2'] = data['V2'].apply(lambda x: encoding_bin(x))
data['V4'] = data['V4'].apply(lambda x: encoding_bin(x))
data['V5'] = data['V5'].apply(lambda x: encoding_bin(x))

In [6]:
data.to_csv('./data/data.csv', index=0)
log.to_csv('./data/log.csv', index=0)